<h1>DS Con: Clustering Backend</h1>

## Importing libraries

In [1]:
import mysql.connector
import getpass

In [2]:
pswd = getpass.getpass('Password:')

Password:········


try:
    cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
    cursor = cnx.cursor(buffered=True)
    cursor.execute("CREATE TABLE pastebindsconwords1 (WORD varchar(1000))")
    cursor.execute("CREATE TABLE pastebindsconwords2 (WORD varchar(1000))")
    cursor.execute("CREATE TABLE pastebindsconwords3 (WORD varchar(1000))")
    cursor.execute("CREATE TABLE pastebindsconwords4 (WORD varchar(1000))")
    cursor.execute("CREATE TABLE pastebindsconwords5 (WORD varchar(1000))")
except mysql.connector.Error as err:
    print(str(err))
else:
    cursor.close()
    cnx.close()
    
try:
    cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
    cursor = cnx.cursor(buffered=True)
    cursor.execute("CREATE TABLE pastebindsconclusters1 (LINK varchar(40), TIME varchar(40), TEXT varchar(60000))")
    cursor.execute("CREATE TABLE pastebindsconclusters2 (LINK varchar(40), TIME varchar(40), TEXT varchar(60000))")
    cursor.execute("CREATE TABLE pastebindsconclusters3 (LINK varchar(40), TIME varchar(40), TEXT varchar(60000))")
    cursor.execute("CREATE TABLE pastebindsconclusters4 (LINK varchar(40), TIME varchar(40), TEXT varchar(60000))")
    cursor.execute("CREATE TABLE pastebindsconclusters5 (LINK varchar(40), TIME varchar(40), TEXT varchar(60000))")
except mysql.connector.Error as err:
    print(str(err))
else:
    cursor.close()
    cnx.close()

## Clustering

In [3]:
import collections
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import time
from signal import signal, SIGPIPE, SIG_DFL
signal(SIGPIPE, SIG_DFL) 

ticker=1
while 1 == 1:
    time.sleep(60)
    print(ticker)
    ticker = ticker+1
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconlarge")
        result = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    pastebin = pd.DataFrame(result,columns = ['link','time','hex content raw','hex content'])
    pastebin['ascii content'] = pastebin['hex content'].apply(lambda x: x.decode("hex"))

    def cluster_texts(texts, clusters=3):
        vectorizer = TfidfVectorizer(max_df=0.5,min_df=0.1)

        tfidf_model = vectorizer.fit_transform(texts)
        km_model = KMeans(n_clusters=clusters)
        km_model.fit(tfidf_model)

        clustering = collections.defaultdict(list)

        for idx, label in enumerate(km_model.labels_):
            clustering[label].append(idx)

        return clustering

    articles = pastebin[['link','time','ascii content']]
    clusters = cluster_texts(pastebin['ascii content'], clusters=5)
    
    for i in range(len(clusters)):
        exec('clusters%s = []') % i
        exec('articles%s = []') % (i+1)
        for j in clusters[i]:
            exec("articles%s.append((articles.iloc[j,:][0],articles.iloc[j,:][1],articles.iloc[j,:][2]))") % (i+1)
            for k in pastebin['ascii content'][j].split():
                exec("clusters%s.append((k,))") % i

    for i in range(1,len(clusters)+1):
        try:
            cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
            cursor = cnx.cursor(buffered=True)
            exec('cursor.execute("TRUNCATE pastebindsconwords%s")') % (i)
            exec('stmt = "INSERT INTO pastebindsconwords%s (WORD) VALUES (%s)"') % (i, "%s")
            exec('cursor.executemany(stmt, clusters%s)') % (i-1)
            cnx.commit()
        except mysql.connector.Error as err:
            print(str(err))
        else:
            cursor.close()
            cnx.close()
            
    for i in range(len(clusters)):
        try:
            cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
            cursor = cnx.cursor(buffered=True)
            exec('cursor.execute("TRUNCATE pastebindsconclusters%s")') % (i+1)
            exec('stmt = "INSERT INTO pastebindsconclusters%s (LINK, TIME, TEXT) VALUES (%s)"') % (i+1, "%s, %s, %s")
            exec('cursor.executemany(stmt, articles%s)') % (i+1)
            cnx.commit()
        except mysql.connector.Error as err:
            print(str(err))
        else:
            cursor.close()
            cnx.close()

1
2


KeyboardInterrupt: 